In [1]:
import os

In [2]:
#!pip install pandas
!{sys.executable} -m pip install --user --upgrade tabula-py==2.9.0

/bin/bash: {sys.executable}: command not found


In [3]:
import pandas as pd
import tabula

In [4]:
file1 = 'KK-SCIVIAS-00003^0055176166^2020-12-28^KIITCH.pdf' # Contains table
file2 = 'KK-SCIVIAS-00004^0054584394^2021-01-17^KIITCH.pdf' # No table

In [5]:
def extract_columns_to_dict(pdf_path, page_numbers='all'):
    """
    Extract tables from a PDF and organize the data into a list of combined dictionaries.
    
    Parameters:
    - pdf_path: Path to the PDF file.
    - page_numbers: Pages to extract tables from ('all' for all pages).
    
    Returns:
    - A list of dictionaries, each representing combined data from the same column across all tables.
    """
    # Extract all tables from the specified pages of the PDF
    tables = tabula.read_pdf(pdf_path, pages=page_numbers, multiple_tables=True, pandas_options={'header': None})
    
    # Initialize a list to collect data from each table
    collected_data = []

    # Iterate through each extracted table
    for table in tables:
        # Iterate through each row in the current table
        for index, row in table.iterrows():
            # Skip rows where the first column is NaN
            if pd.isna(row[0]):
                continue
            
            # Create a dictionary for the current row, excluding the first column
            row_dict = {f"col{col_idx}": row[col_idx] for col_idx in range(1, len(row))}
            
            # Use the value of the first column as the key for the current row's data
            collected_data.append({row[0]: row_dict})

    if not collected_data:
        print("No tables found")
        return []

    column_keys = next(iter(collected_data[0].values())).keys()
    
    # Initialize a list for data dictionaries
    combined_list = []

    # Combine data across the same column from all dictionaries
    for key in column_keys:
        combined_dict = {d_key: d_val[key] for data in collected_data for d_key, d_val in data.items()}
        combined_list.append(combined_dict)
    
    return combined_list


In [6]:
# data = extract_columns_to_dict(file1)

# for item in data:
#     print(item)